In [ ]:
using JuMP, Ipopt
using Gurobi

In [ ]:
using MAT
using DataFrames

# Load the MAT file
mat_case = matread("case14.mat")

# Extract the bus data
df_bus = DataFrame(mat_case["ans"]["bus"],:auto)

# Rename the columns
rename!(df_bus, Symbol.([:bus_i, :type, :Pd, :Qd, :Gs, :Bs, :area, :Vm, :Va, :baseKV, :zone, :Vmax, :Vmin]))

# Convert data types
df_bus.bus_i = Int.(df_bus.bus_i)
df_bus.type = Int.(df_bus.type)

# Extract the generator data
df_gen = DataFrame(mat_case["ans"]["gen"],:auto)

# Rename the columns
rename!(df_gen, Symbol.([:bus, :Pg, :Qg, :Qmax, :Qmin, :Vg, :mBase, :status, :Pmax, :Pmin, :Pc1, :Pc2, :Qc1min, :Qc1max, :Qc2min, :Qc2max, :ramp_agc, :ramp_10, :ramp_30, :ramp_q, :apf]))

# Convert data types
df_gen.bus = Int.(df_gen.bus)

# Extract the branch data
df_branch = DataFrame(mat_case["ans"]["branch"],:auto)

# Rename the columns
rename!(df_branch, Symbol.([:fbus, :tbus, :r, :x, :b, :rateA, :rateB, :rateC, :ratio, :angle, :status, :angmin, :angmax]))

# Convert data types
df_branch.fbus = Int.(df_branch.fbus)
df_branch.tbus = Int.(df_branch.tbus)

In [ ]:
# UBase: Base voltage in volts (V)
UBase = df_bus[1, :baseKV] * 1e3

# SBase: Base apparent power in MVA
SBase = df_gen[1, :mBase]

# P_gen_max and P_gen_min: Maximum and Minimum Active Power of Generators in p.u.
P_gen_max = df_gen[:, :Pmax] ./ SBase
P_gen_min = df_gen[:, :Pmin] ./ SBase

# Q_gen_max and Q_gen_min: Maximum and Minimum Reactive Power of Generators in p.u.
Q_gen_max = df_gen[:, :Qmax] ./ SBase
Q_gen_min = df_gen[:, :Qmin] ./ SBase

# r_branch, x_branch, and b_branch: Resistance, Reactance, and Susceptance of Branches
r_branch = df_branch[:, :r]
x_branch = df_branch[:, :x]
b_branch = df_branch[:, :b]

# Pd_node and Qd_node: Active and Reactive Power Demands at Nodes in p.u.
Pd_node = df_bus[:, :Pd] ./ SBase
Qd_node = df_bus[:, :Qd] ./ SBase

# Gs_node and Bs_node: Shunt Conductance and Susceptance at Nodes in p.u.
Gs_node = df_bus[:, :Gs] ./ SBase
Bs_node = df_bus[:, :Bs] ./ SBase

# I_max_branch: Maximum Current Rating of Branches in p.u.
I_max_branch = df_branch[:, :rateA] ./ SBase

# V_max_node and V_min_node: Maximum and Minimum Voltage Magnitudes at Nodes
V_max_node = df_bus[:, :Vmax]
V_min_node = df_bus[:, :Vmin]

# Branch_num, Power_node_num, and Gen_node_num: Number of Branches, Power Nodes, and Generator Nodes
Branch_num = size(df_branch, 1)
Power_node_num = size(df_bus, 1)
Gen_node_num = size(df_gen, 1)

# Branch_nodes: Extracting the 'fbus' and 'tbus' columns as an integer array
Branch_nodes = Matrix(df_branch[:, [:fbus, :tbus]])

# Gen_nodes: Extracting the 'bus' column from generator data as an integer array
Gen_nodes = Int.(df_gen[:, :bus])

# Branch_status: Extracting the 'status' column from branch data
Branch_status = df_branch[:, :status]

# f_bus and t_bus: Extracting 'fbus' and 'tbus' columns as integer arrays
f_bus = Int.(df_branch[:, :fbus])
t_bus = Int.(df_branch[:, :tbus])

# Find the slack bus whose type is 3
slack_bus = Int.(df_bus[df_bus[:, :type] .== 3, :bus_i])

In [ ]:
using LinearAlgebra

# Calculate the complex admittance for branches
y_branch = Branch_status ./ (r_branch + 1im .* x_branch)

# Calculate line charging susceptance
b_branch = b_branch .* Branch_status

# Default tap ratio is 1
Tap_ratio = ones(Complex{Float64}, Branch_num)

# Update the tap ratio if it's not 0
nonzero_ratio_indices = findall(x -> x != 0, df_branch[:, :ratio])
nonzero_ratios = df_branch[nonzero_ratio_indices, :ratio]
Tap_ratio[nonzero_ratio_indices] = nonzero_ratios

# Add phase shifters
Tap_ratio .= Tap_ratio .* exp.(1im .* 3.14 / 180 .* df_branch[:, :angle])

# Calculate Ytt, Yff, Yft, Ytf
Ytt = y_branch .+ 1im .* b_branch ./ 2
Yff = Ytt ./ (Tap_ratio .* conj(Tap_ratio))
Yft = -y_branch ./ conj(Tap_ratio)
Ytf = -y_branch ./ Tap_ratio

# Shunt admittance
Ysh = Gs_node .+ 1im .* Bs_node

# Initialize Ybus matrix
Ybus = zeros(Complex{Float64}, Power_node_num, Power_node_num)


for l in 1:Branch_num
    Ybus[f_bus[l], f_bus[l]] += Yff[l]
    Ybus[t_bus[l], t_bus[l]] += Ytt[l]
    Ybus[f_bus[l], t_bus[l]] += Yft[l]
    Ybus[t_bus[l], f_bus[l]] += Ytf[l]
end

# Add shunt admittance to the diagonal elements of Ybus
for i in 1:Power_node_num
    Ybus[i, i] += Ysh[i]
end

# Extract the real and imaginary parts of the Ybus matrix
G_bus = real(Ybus)
B_bus = imag(Ybus)

In [ ]:
function Node_Incidence_Matrix(Nodes_Counts, Line_dat)
    # Initialize the node-incidence matrix
    N_Matrix = zeros(Int, Nodes_Counts, Nodes_Counts)
    println("??")
    # Iterate over each line
    for line in 1:size(Line_dat)[1]
        F_Bus, T_Bus = Int(Line_dat[line,1]), Int(Line_dat[line,2])
        println("Nodes_Counts: $Nodes_Counts, F_Bus: $F_Bus, T_Bus: $T_Bus")  # Debugging line
        # Mark the nodes as connected
        N_Matrix[F_Bus, T_Bus] = 1
        N_Matrix[T_Bus, F_Bus] = 1  # Since the connection is bidirectional
    end

    return N_Matrix
end


function Incidence_matrix(Nodes_Counts, Line_dat)
    F_Bus = Line_dat[:, 1]
    T_Bus = Line_dat[:, 2]
    # Initialize A_Matrix
    A_Matrix = zeros(Int, Nodes_Counts, length(F_Bus))
    # Populate A_Matrix
    for i in 1:length(F_Bus)
        A_Matrix[F_Bus[i], i] = 1
        A_Matrix[T_Bus[i], i] = -1
    end
    return A_Matrix
end

function Inject_matrix(Nodes_Counts, Line_dat)
    # Extracting 'from' buses from Line_dat
    F_Bus = Line_dat[:, 1]
    # Initialize A_Matrix
    A_Matrix = zeros(Int, Nodes_Counts, length(F_Bus))
    # Populate A_Matrix
    for i in 1:length(F_Bus)
        A_Matrix[F_Bus[i], i] = 1
    end
    return A_Matrix
end

function Gen_matrix(Nodes_Counts, Gen_dat)
    # Initialize A_Matrix
    A_Matrix = zeros(Int, Nodes_Counts, length(Gen_dat))
    # Populate A_Matrix
    for i in 1:length(Gen_dat)
        A_Matrix[Gen_dat[i], i] = 1
    end
    return A_Matrix
end


In [ ]:
Node_incidence = Node_Incidence_Matrix(Power_node_num, Branch_nodes)

# Obtain the incidence matrix
S_incidence = Incidence_matrix(Power_node_num, Branch_nodes)

# Obtain the injection matrix
S_inject = Inject_matrix(Power_node_num, Branch_nodes)

# Obtain the generator matrix
G_matrix = Gen_matrix(Power_node_num, Gen_nodes)

In [ ]:
# Create a dictionary to store neighbors
neighbors_dict = Dict{Int, Set{Int}}()
gen_dict = Dict{Int, Set{Int}}()

# Iterate over nodes (1 to 5 in this example)
for node in 1:size(Node_incidence, 1)
    neighbors = Set{Int}()
    push!(neighbors, node)
    # Iterate over columns of the Node_incidence matrix
    for neighbor_node in 1:size(Node_incidence, 2)
        if Node_incidence[node, neighbor_node] == 1
            push!(neighbors, neighbor_node)
        end
    end
    
    neighbors_dict[node] = neighbors
end

for node in 1:size(G_matrix, 1)
    neighbors = Set{Int}()
    
    # Iterate over columns of the Node_incidence matrix
    for neighbor_node in 1:size(G_matrix, 2)
        if G_matrix[node, neighbor_node] == 1
            push!(neighbors, neighbor_node)
        end
    end
    
    gen_dict[node] = neighbors
end

In [ ]:
head = Dict{Int, Array{Int, 1}}()
tail = Dict{Int, Array{Int, 1}}()

for l in 1:Power_node_num
    head[l]=[]
    tail[l]=[]
end

# Populate the dictionaries
for (idx, line) in enumerate(eachrow(Branch_nodes))
    h, t = line
    
    # For head
    if haskey(head, h)
        push!(head[h], idx)
    else
        head[h] = [idx]
    end
    
    # For tail
    if haskey(tail, t)
        push!(tail[t], idx)
    else
        tail[t] = [idx]
    end
end

In [ ]:
pi=3.1415926
model = Model(optimizer_with_attributes(Gurobi.Optimizer))
#set_attribute(model, "max_cpu_time", 60.0)
#set_attribute(model, "max_iter", 10000)
#set_attribute(model, "print_level", 1)

# Add variables
# Voltage magnitude and angle (now v is no longer the square of voltage magnitude)
@variables model begin
    v_node_voltage[1:Power_node_num] >= 0
    -pi<=theta_node_voltage[1:Power_node_num] <=pi
end

@variables model begin
    P_gen_power[1:Gen_node_num]
    Q_gen_power[1:Gen_node_num]
end

@variables model begin
    P_flow_power[1:Branch_num]
    Q_flow_power[1:Branch_num]
end

# Set objective
@objective(model, Min, sum(P_gen_power[g] for g in 1:Gen_node_num))

In [ ]:
# Add constraints
for i in 1:Power_node_num
    # Polar power-voltage formulation
    @constraint(model, 
        sum( P_gen_power[g] for g in gen_dict[i]) - Pd_node[i] ==
        sum(P_flow_power[j] for j in head[i]) - sum(P_flow_power[j] for j in tail[i])
    )
end

for i in 1:Power_node_num
    # Polar power-voltage formulation
    @constraint(model, 
        sum( Q_gen_power[g] for g in gen_dict[i]) - Qd_node[i] ==
        sum(Q_flow_power[j] for j in head[i]) - sum(Q_flow_power[j] for j in tail[i])
    )
    
    @constraint(model, v_node_voltage[i] <= V_max_node[i]*V_max_node[i]-0.00005)
    @constraint(model, v_node_voltage[i] >= V_min_node[i]*V_min_node[i]+0.00005)
end
    
for l in 1:Branch_num
    i = Branch_nodes[l, 1]
    j = Branch_nodes[l, 2]
     @constraint(model, 
         v_node_voltage[i]-v_node_voltage[j] ==
        2*(r_branch[l]*P_flow_power[l] +x_branch[l]*Q_flow_power[l])
     )


end

for l in 1:Branch_num
    index_i = Branch_nodes[l, 1]
    index_j = Branch_nodes[l, 2]
    # Branch flow limit
    if I_max_branch[l] > 0.01
        @NLconstraint(model, P_branch_flow[l]^2 + Q_branch_flow[l]^2 <= I_max_branch[l]^2)
    end
end

for g in 1:Gen_node_num
    @constraint(model, P_gen_power[g] <= P_gen_max[g])
    @constraint(model, Q_gen_power[g] <= Q_gen_max[g])
    @constraint(model, P_gen_power[g] >= P_gen_min[g])
    @constraint(model, Q_gen_power[g] >= Q_gen_min[g])
end

In [ ]:
# Solve the model
optimize!(model)

In [ ]:
voltage_initial=value.(v_node_voltage)
P_gen_initial = value.(P_gen_power)
Q_gen_initial = value.(Q_gen_power)

In [ ]:
# Create a new JuMP model
pi=3.1415926
model = Model(optimizer_with_attributes(Ipopt.Optimizer))
set_attribute(model, "max_cpu_time", 60.0)
set_attribute(model, "max_iter", 10000)
set_attribute(model, "print_level", 1)

# Add variables
# Voltage magnitude and angle (now v is no longer the square of voltage magnitude)
@variables model begin
    v_node_voltage[1:Power_node_num] >= 0
    -pi/2<=theta_node_voltage[1:Power_node_num] <=pi/2
end

for i in 1:Power_node_num
    set_start_value(v_node_voltage[i], sqrt(voltage_initial[i]))
end

@variables model begin
    P_gen_power[1:Gen_node_num]
    Q_gen_power[1:Gen_node_num]
end

for i in 1:Gen_node_num
    set_start_value(P_gen_power[i], P_gen_initial[i])
    set_start_value(Q_gen_power[i], Q_gen_initial[i])
end


# Set objective
@objective(model, Min, sum(P_gen_power[g] for g in 1:Gen_node_num))

In [ ]:
# Add constraints
for i in 1:Power_node_num
    # Polar power-voltage formulation
    @NLconstraint(model, 
        sum( P_gen_power[g] for g in gen_dict[i]) - Pd_node[i] ==
        v_node_voltage[i] * sum(v_node_voltage[j] *
        (G_bus[i, j] * cos(theta_node_voltage[i] - theta_node_voltage[j]) +
        B_bus[i, j] * sin(theta_node_voltage[i] - theta_node_voltage[j])) for j in neighbors_dict[i])
    )
end
    
for i in 1:Power_node_num
     @NLconstraint(model, 
         sum(Q_gen_power[g] for g in gen_dict[i]) - Qd_node[i] ==
         v_node_voltage[i] * sum(v_node_voltage[j] *1*
         (G_bus[i, j] * sin(theta_node_voltage[i] - theta_node_voltage[j]) -
         B_bus[i, j] * cos(theta_node_voltage[i] - theta_node_voltage[j])) for j in neighbors_dict[i])
     )

    # Node voltage limit
    @constraint(model, v_node_voltage[i] <= V_max_node[i])
    @constraint(model, v_node_voltage[i] >= V_min_node[i])
end

for l in 1:Branch_num
    index_i = Branch_nodes[l, 1]
    index_j = Branch_nodes[l, 2]
    # Branch flow limit
    if I_max_branch[l] > 0.01
        @NLconstraint(model, P_branch_flow[l]^2 + Q_branch_flow[l]^2 <= I_max_branch[l]^2)
    end
end

for g in 1:Gen_node_num
    @constraint(model, P_gen_power[g] <= P_gen_max[g])
    @constraint(model, Q_gen_power[g] <= Q_gen_max[g])
    @constraint(model, P_gen_power[g] >= P_gen_min[g])
    @constraint(model, Q_gen_power[g] >= Q_gen_min[g])
end

# Slack bus voltage angle equal to 0
@constraint(model, theta_node_voltage[slack_bus[1]] == 0)


# Solve the model
optimize!(model)